# Workflow

在本节中，你将学习如何使用 `GraphFlow`（简称 “flow”）创建一个多智能体工作流。GraphFlow 采用结构化执行方式，可以精确控制各个代理之间的交互流程，以完成特定任务。

## 什么是 GraphFlow？

GraphFlow 是 AutoGen 中的一种团队（Team）结构，基于有向图（DiGraph）来控制代理（Agent）之间的执行流程。它支持以下复杂行为：

- 顺序执行（Sequential）

- 并行执行（Parallel）

- 条件分支（Conditional）

- 循环（Looping）

> langgraph 可以类比学习


## 何时应该使用 GraphFlow？
你应该在以下场景中使用 GraphFlow：

- 你需要严格控制代理的执行顺序

- 不同结果需要触发不同后续步骤

- 任务是一个多步骤流程，并可能包含条件分支或循环

- 需要确定性执行，而非随意聊天风格

👉 如果你的任务是非结构化对话，建议优先使用更简单的团队结构，如：

- `RoundRobinGroupChat`

- `SelectorGroupChat`

在任务变得复杂或需要结构控制时，再切换到 `GraphFlow`。




## 配置 Client

In [3]:
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient

load_dotenv()
siliconflow_api_key = os.getenv("SILICONFLOW_API_KEY") # 读取你的 OPENAI API key

# 初始化 OpenAIChatCompletionClient 客户端，连接到硅基流动平台的 Qwen3-8B 模型
model_client = OpenAIChatCompletionClient(
    model="Qwen/Qwen3-14B",                # 指定要调用的模型名称，硅基流动平台上 Qwen 3-8B 模型
    base_url="https://api.siliconflow.cn/v1",  # 硅基流动平台的 API 访问地址
    api_key=siliconflow_api_key,  # 你的 API 密钥
    model_info={                        
        "family": "qwen",              
        "context_length": 8192,        
        "max_output_tokens": 2048,     
        "tool_choice_supported": True, 
        "tool_choice_required": False,  
        "structured_output": True,     
        "vision": False,                
        "function_calling": True,      
        "json_output": True           
    },
)

## Creating and Running a Flow

`DiGraphBuilder` 是一个流式接口工具，能够让你轻松构建用于工作流的执行图。它支持构建：

- 顺序链 Sequential chains
- 并行分支 Parallel fan-outs
- 条件分支 Conditional branching
- 带安全退出条件的循环 Loops with safe exit conditions

图中的每个节点代表一个智能体，边定义了允许的执行路径。边还可以根据智能体的消息设置执行条件。

参考：https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.DiGraphBuilder


### Sequential Flow

我们将从创建一个简单的工作流开始：一个写作者起草一段文字，随后一个审阅者提供反馈。该流程在审阅者完成评论后结束。

注意，流程会自动计算图中的所有源节点和叶节点，执行从所有源节点开始，当没有剩余节点可执行时，流程结束。

![绘图2.png](resources\asset\绘图2.png)


In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_ext.models.openai import OpenAIChatCompletionClient

# 创建 OpenAI 模型客户端
client = model_client

# 创建写作智能体
writer = AssistantAgent(
    "writer",
    model_client=client,
    system_message="Draft a short paragraph on climate change."
)

# 创建审稿智能体
reviewer = AssistantAgent(
    "reviewer",
    model_client=client,
    system_message="Review the draft and suggest improvements."
)

# 构建有向图（工作流）
builder = DiGraphBuilder()
builder.add_node(writer).add_node(reviewer) # 添加智能体节点, 注意添加的时候直接输入 智能体对象即可，该方法会自动读取智能体的名称
builder.add_edge(writer, reviewer)  # writer 执行后交给 reviewer


# 构建并校验图结构
graph = builder.build()

# 创建 GraphFlow 工作流
flow = GraphFlow([writer, reviewer], graph=graph)

In [5]:
# 如果在脚本中运行，请用 asyncio.run(...) 包裹主函数
# 推荐用 Console(flow.run_stream(...)) 获得更好的控制台格式化输出

stream = flow.run_stream(task="Write a short paragraph about climate change.")
async for event in stream:  # type: ignore
    print(event)

# 或者更推荐这样写，输出更美观：
# await Console(flow.run_stream(task="Write a short paragraph about climate change."))

source='user' models_usage=None metadata={} content='Write a short paragraph about climate change.' type='TextMessage'
source='writer' models_usage=None metadata={} content='\nOkay, the user asked me to draft a short paragraph on climate change. Let me start by understanding what they need. They probably want a concise overview that\'s informative but not too technical. Maybe for a school project, a presentation, or just a quick explanation.\n\nFirst, I should define climate change in simple terms. Mention the primary cause, which is human activities like burning fossil fuels leading to greenhouse gases. Then, the effects: rising temperatures, extreme weather, sea level rise, impact on ecosystems and human societies. Need to include both causes and consequences. Also, maybe touch on the urgency and solutions briefly, like reducing emissions and transitioning to renewable energy. Should keep it balanced, not too alarmist but highlighting the importance of action. Let me check the key po

### Parallel Flow with Join

我们将创建一个稍微复杂的工作流，流程如下：

1. 写作者撰写一段文字；

2. 然后流程并行分发给两位编辑：

3. 一位负责语法修改；

4. 一位负责文风优化；

5. 最后，两位编辑的修改结果被传递给最终审阅者进行汇总与评审。

该流程从写作者节点启动，随后并行执行两个编辑节点，最后在审阅者节点完成汇总并终止。

In [6]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# 创建 OpenAI 模型客户端
client = model_client

# 创建写作智能体
writer = AssistantAgent("writer", model_client=client, system_message="Draft a short paragraph on climate change.")

# 创建两个编辑智能体
editor1 = AssistantAgent("editor1", model_client=client, system_message="Edit the paragraph for grammar.")
editor2 = AssistantAgent("editor2", model_client=client, system_message="Edit the paragraph for style.")

# 创建最终审稿智能体
final_reviewer = AssistantAgent(
    "final_reviewer",
    model_client=client,
    system_message="Consolidate the grammar and style edits into a final version.",
)

# 构建工作流图
builder = DiGraphBuilder()
builder.add_node(writer).add_node(editor1).add_node(editor2).add_node(final_reviewer)

# 从 writer 节点分发到 editor1 和 editor2（并行）
builder.add_edge(writer, editor1)
builder.add_edge(writer, editor2)

# 两个编辑节点的结果汇总到 final_reviewer
builder.add_edge(editor1, final_reviewer)
builder.add_edge(editor2, final_reviewer)

# 构建并校验图结构
graph = builder.build()

# 创建 GraphFlow 工作流
flow = GraphFlow(
    participants=builder.get_participants(),
    graph=graph,
)

# 运行工作流，并将消息流式输出到控制台
await Console(flow.run_stream(task="Write a short paragraph about climate change."))

---------- TextMessage (user) ----------
Write a short paragraph about climate change.
---------- ThoughtEvent (writer) ----------

Okay, the user asked me to write a short paragraph about climate change. Let me start by understanding the key points they might want covered. Climate change is a broad topic, so I need to keep it concise but informative.

First, I should mention the primary cause, which is human activities like burning fossil fuels leading to greenhouse gas emissions. Then, the effects: rising temperatures, extreme weather events, melting ice, and sea level rise. Also, the impact on ecosystems and human societies. Maybe include something about the need for global action, like reducing emissions and transitioning to renewable energy. I should make sure the paragraph flows logically from cause to effect to solutions. Let me check the latest data to ensure accuracy, but since it's a short paragraph, I don't need too much detail. Avoid jargon to keep it accessible. Need to hi

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short paragraph about climate change.', type='TextMessage'), ThoughtEvent(source='writer', models_usage=None, metadata={}, content="\nOkay, the user asked me to write a short paragraph about climate change. Let me start by understanding the key points they might want covered. Climate change is a broad topic, so I need to keep it concise but informative.\n\nFirst, I should mention the primary cause, which is human activities like burning fossil fuels leading to greenhouse gas emissions. Then, the effects: rising temperatures, extreme weather events, melting ice, and sea level rise. Also, the impact on ecosystems and human societies. Maybe include something about the need for global action, like reducing emissions and transitioning to renewable energy. I should make sure the paragraph flows logically from cause to effect to solutions. Let me check the latest data to ensure accuracy, but since

## Message Filtering

Execution Graph vs. Message Graph

在 `GraphFlow` 中，执行图是通过 `DiGraph` 定义的，用于控制各个智能体的执行顺序。然而，执行图**并不控制**一个智能体从其他智能体接收哪些消息。默认情况下，图中的**所有消息**都会被发送给**所有智能体**。

**消息过滤**是一个独立的功能，它允许你为每个智能体筛选其接收的消息，从而使其模型上下文中仅包含**相关信息**。一组消息过滤规则共同定义了流程中的**消息图**。

指定消息图有助于：

* 减少幻觉（hallucinations）
* 控制上下文内存负载
* 使智能体聚焦于与其任务相关的信息

> 简单来说，执行图 Execution Graph 是定义工作流执行规则，消息图 Message Graph 定义了工作流的消息传递规则。

你可以使用 `MessageFilterAgent`，结合 `MessageFilterConfig` 和 `PerSourceFilter` 来定义这些规则。


下面是一个消息图的示例

![绘图1.png](resources\asset\绘图1.png)

In [7]:
from autogen_agentchat.agents import AssistantAgent, MessageFilterAgent, MessageFilterConfig, PerSourceFilter
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# 创建模型客户端
client = model_client

# 创建智能体
researcher = AssistantAgent(
    "researcher", model_client=client, system_message="Summarize key facts about climate change."
)
analyst = AssistantAgent("analyst", model_client=client, system_message="Review the summary and suggest improvements.")
presenter = AssistantAgent(
    "presenter", model_client=client, system_message="Prepare a presentation slide based on the final summary."
)

# 应用消息过滤器：只让 analyst 看到 researcher 的最后一条消息
filtered_analyst = MessageFilterAgent(
    name="analyst",
    wrapped_agent=analyst,
    filter=MessageFilterConfig(per_source=[PerSourceFilter(source="researcher", position="last", count=1)]),
)

# 只让 presenter 看到 analyst 的最后一条消息
filtered_presenter = MessageFilterAgent(
    name="presenter",
    wrapped_agent=presenter,
    filter=MessageFilterConfig(per_source=[PerSourceFilter(source="analyst", position="last", count=1)]),
)

# 构建工作流图
builder = DiGraphBuilder()
builder.add_node(researcher).add_node(filtered_analyst).add_node(filtered_presenter)
builder.add_edge(researcher, filtered_analyst).add_edge(filtered_analyst, filtered_presenter)

# 创建 GraphFlow 工作流
flow = GraphFlow(
    participants=builder.get_participants(),
    graph=builder.build(),
)

# 运行工作流，并将消息流式输出到控制台
await Console(flow.run_stream(task="Summarize key facts about climate change."))

---------- TextMessage (user) ----------
Summarize key facts about climate change.
---------- ThoughtEvent (researcher) ----------

Okay, so I need to summarize key facts about climate change. Let me start by recalling what I know. Climate change refers to long-term changes in temperature and weather patterns, right? It's mostly caused by human activities, especially the burning of fossil fuels which releases greenhouse gases. Carbon dioxide is a big one, along with methane and nitrous oxide. These gases trap heat in the atmosphere, leading to the greenhouse effect.

I remember that the Earth's average temperature has been rising. Scientists have data from ice cores, temperature records, and satellites. The past few decades have seen significant increases, and there's a lot of consensus that this is mainly due to human activities. The Intergovernmental Panel on Climate Change (IPCC) reports are important here, but I need to check if that's accurate.

Then there's the impact on ecosyste

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Summarize key facts about climate change.', type='TextMessage'), ThoughtEvent(source='researcher', models_usage=None, metadata={}, content="\nOkay, so I need to summarize key facts about climate change. Let me start by recalling what I know. Climate change refers to long-term changes in temperature and weather patterns, right? It's mostly caused by human activities, especially the burning of fossil fuels which releases greenhouse gases. Carbon dioxide is a big one, along with methane and nitrous oxide. These gases trap heat in the atmosphere, leading to the greenhouse effect.\n\nI remember that the Earth's average temperature has been rising. Scientists have data from ice cores, temperature records, and satellites. The past few decades have seen significant increases, and there's a lot of consensus that this is mainly due to human activities. The Intergovernmental Panel on Climate Change (IPCC) rep

## Advanced Example: Conditional Loop + Filtered Summary

本示例综合展示了以下内容的结合：

- **条件循环(Conditional Loop)：** 在 `generator` 和 `reviewer` 之间的一个循环（当 `reviewer` 回复“APPROVE”时退出）

- **过滤摘要(iltered Summary)：** 一个 `summarizer` 智能体，它仅接收用户的首次输入和 `reviewer` 的最后一条消息

In [16]:
from autogen_agentchat.agents import AssistantAgent, MessageFilterAgent, MessageFilterConfig, PerSourceFilter
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from typing import Dict

# 定义全局状态存储迭代次数
workflow_state: Dict[str, int] = {"iteration": 0}

# 定义迭代次数检查函数
def check_iteration_limit(msg):
    workflow_state["iteration"] += 1
    return workflow_state["iteration"] <= 2  # 最大迭代5次
    
# 初始化模型客户端
model_client = model_client

# 创建各角色智能体
generator = AssistantAgent(
    "generator",
    model_client=model_client,
    system_message="Generate a list of creative ideas."
)
reviewer = AssistantAgent(
    "reviewer",
    model_client=model_client,
    system_message="Review ideas and provide feedbacks, or just 'APPROVE' for final approval.",
)
summarizer_core = AssistantAgent(
    "summary",
    model_client=model_client,
    system_message="Summarize the user request and the final feedback."
)

# 创建带消息过滤的 summary 智能体，只看 user 的首条消息和 reviewer 的最后一条消息
filtered_summarizer = MessageFilterAgent(
    name="summary",
    wrapped_agent=summarizer_core,
    filter=MessageFilterConfig(
        per_source=[
            PerSourceFilter(source="user", position="first", count=1),
            PerSourceFilter(source="reviewer", position="last", count=1),
        ]
    ),
)

# 构建带条件循环的工作流图
builder = DiGraphBuilder()
builder.add_node(generator).add_node(reviewer).add_node(filtered_summarizer)
builder.add_edge(generator, reviewer)
# 如果 reviewer 回复中包含 "APPROVE"，则进入 summary，否则回到 generator 继续生成
builder.add_edge(reviewer, filtered_summarizer, condition='"APPROVE" in msg.to_model_text()')
builder.add_edge(reviewer, generator, condition='"APPROVE" not in msg.to_model_text() and check_iteration_limit(msg)')
builder.set_entry_point(generator)  # 显式设置入口节点

graph = builder.build()

# 创建 GraphFlow 工作流
flow = GraphFlow(
    participants=builder.get_participants(),
    graph=graph,
)

# 运行工作流，并将消息流式输出到控制台
await Console(flow.run_stream(task="Brainstorm ways to reduce plastic waste."))

---------- TextMessage (user) ----------
Brainstorm ways to reduce plastic waste.
---------- ThoughtEvent (generator) ----------

Okay, I need to come up with creative ideas to reduce plastic waste. Let me start by thinking about the different areas where plastic is commonly used and how those can be addressed. 

First, everyday items like packaging, bottles, and bags. Maybe people can use reusable containers instead of single-use plastics. I know some stores offer incentives for bringing your own bags, so maybe expanding that idea? Like, maybe a rewards program for using reusable containers for takeout. Oh, and instead of plastic packaging, using paper or biodegradable materials. But those might not be as effective if not composted properly.

Then there's the food industry. Stores could use bulk bins for dry goods, which customers can bring their own containers for. That's already done in some places, but maybe more creative ways? What about edible packaging? I've heard of some compan

Error processing publish message for GraphManager_bb84edf6-b10c-4dc1-bd26-4c2ba1faaa38/bb84edf6-b10c-4dc1-bd26-4c2ba1faaa38
Traceback (most recent call last):
  File "D:\anaconda\envs\autogen\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 604, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\envs\autogen\Lib\site-packages\autogen_core\_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\envs\autogen\Lib\site-packages\autogen_agentchat\teams\_group_chat\_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\envs\autogen\Lib\site-packages\autogen_core\_routed_agent.py", line 485, in on_message_impl
    return await h(self, message, ctx)
           ^^^^^^^^^^^^^^^^^^^^


Okay, let's take a look at this list of 30 ideas for reducing plastic waste. The user has organized them into themes like innovation, policy, education, community action, etc. I need to review these ideas critically and provide feedback. Let me start by going through each one and assessing their feasibility, impact, and any potential issues.

First idea: Edible packaging with seaweed or cornstarch. That sounds promising. It's a novel approach, but I wonder about the cost and scalability. Are these materials suitable for all types of packaging? Maybe some foods need more protection, so the adoption might be limited.

Next, Plastic-Neutral Certification. It's a good incentive for companies. But how effective is offsetting? If companies just fund clean-ups instead of reducing their own use, does that really help? There might be a risk of greenwashing if the certification isn't strictly regulated.

Smart Recycling with AI and Robotics. This could definitely improve recycling efficiency. H

RuntimeError: RuntimeError: Condition not met for node reviewer. Content: 

**Feedback on the "Reduce Plastic Waste" Ideas List**  
This is a comprehensive and creative list of strategies, blending technology, policy, community engagement, and education. Below is an organized review with constructive feedback to enhance feasibility, impact, and alignment with real-world challenges:

---

### **Strengths**  
- **Diversity of Approaches**: The ideas span individual behavior, corporate accountability, technological innovation, and systemic change, addressing plastic waste from multiple angles.  
- **Innovation & Creativity**: Concepts like edible packaging, blockchain tracking, and AR filters are forward-thinking and could inspire widespread adoption.  
- **Community Focus**: Initiatives like "plastic-free swaps," clean-up drives, and tool libraries prioritize local engagement, which is key to long-term behavioral change.  
- **Interdisciplinary Potential**: Combines tech (AI, VR), art, e

# (Todo)

`builder.set_entry_point(agent)` 当图的执行顺序不好直接看出来的时候，可以通过这个方法设置图的入口点，然后通过 `builder.run()` 运行图。即，入口可能是个循环。

`builder.get_participants()`

`add_conditional_edges(source, condition_to_target)`

- 复杂的环形执行图
- 并行的复杂环形图
- 一些常用的工作流代码

## Reference


1. https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/graph-flow.html
2. https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.GraphFlow